# Web Scraping Python - éviter la détection

_Ce café python est la suite du précédent café python sur le web scraping et suppose donc que vous savez au moins utiliser requests / selenium et Beautifulsoup_

__Au programme : comment éviter la détection avec__
- les User-Agents et autres headers
- les Proxies

# User-Agents


## Qu'est-ce que c'est ?

Le `User-Agent` est un des nombreux `headers` du protocole HTTP, globalement un des paramètres qui est envoyé par votre navigateur lors qu'il demande (requête) une page Web.

On peut observer le détail en mode développeur (F12) sur un navigateur.


## Pourquoi c'est important ?

Le header `User-Agent` est souvent utilisé pour la détection de bots et donc bloquer le scraping.

Exemple ci-dessous de ce qu'on observe si l'on utilise le module `requests` de python : 


In [1]:
import requests # à installer avec 'pip install requests'
response = requests.get("https://httpbin.org/user-agent")
response.json()

{'user-agent': 'python-requests/2.26.0'}

## Comment le changer ?

In [2]:
headers = {"user-agent": "toto"}
response = requests.get("https://httpbin.org/user-agent", headers=headers)
response.json()

{'user-agent': 'toto'}

__Liens utiles__
- http://www.useragentstring.com/pages/useragentstring.php
- https://developer.chrome.com/docs/multidevice/user-agent/
- https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent

## Autres headers

### Header du module requests

In [3]:
response = requests.get("https://httpbin.org/headers")
response.json()

{'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.26.0',
  'X-Amzn-Trace-Id': 'Root=1-61d1cdf6-6bb15674640f7fb0716c74a0'}}

### Headers de Firefox

In [4]:
import yaml

with open("headers.yml") as f_headers:
    browser_headers = yaml.safe_load(f_headers)
browser_headers["Firefox"]

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
 'Connection': 'keep-alive',
 'DNT': '1',
 'Upgrade-Insecure-Requests': '1',
 'Sec-Fetch-Dest': 'document',
 'Sec-Fetch-Mode': 'navigate',
 'Sec-Fetch-Site': 'cross-site',
 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0'}

In [5]:
response = requests.get("https://httpbin.org/headers", headers=browser_headers["Firefox"])
response.json()

{'headers': {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
  'Accept-Encoding': 'gzip, deflate, br',
  'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
  'Dnt': '1',
  'Host': 'httpbin.org',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'cross-site',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0',
  'X-Amzn-Trace-Id': 'Root=1-61d1ce6a-328d62570c75a9013ae1395a'}}

# Proxies

## Pourquoi utiliser des proxies ?

Tout simplement parce que cela permet de changer l'adresse ip dont provient la requête et donc de faire croire au site que l'on est une personne différente.


## Comment utiliser un proxy ?


In [6]:
proxies = {
    "http": "http://77.86.31.251:8080",
    "https": "http://77.86.31.251:8080",
}
response = requests.get("https://httpbin.org/ip", proxies=proxies)
response.json()

{'origin': '77.86.31.251'}

## Comment changer automatiquement de proxy ?

https://free-proxy-list.net/

In [12]:
response = requests.get("https://free-proxy-list.net/")

In [13]:
import pandas as pd
proxy_list = pd.read_html(response.text)[0]
proxy_list["url"] = "http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
proxy_list.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,url
0,90.45.5.27,80,FR,France,elite proxy,NaN,no,1 min ago,http://90.45.5.27:80
1,94.23.91.209,80,PL,Poland,elite proxy,NaN,no,1 min ago,http://94.23.91.209:80
2,43.224.10.37,6666,IN,India,elite proxy,NaN,no,1 min ago,http://43.224.10.37:6666
3,191.252.177.170,8118,BR,Brazil,elite proxy,NaN,no,1 min ago,http://191.252.177.170:8118
4,200.103.102.18,80,BR,Brazil,anonymous,yes,no,1 min ago,http://200.103.102.18:80


In [14]:
# on copie ici avec pd.DataFrame pour pouvoir ajouter proprement une colonne ensuite
https_proxies = proxy_list[proxy_list["Https"] == "yes"]
https_proxies.count()

IP Address      147
Port            147
Code            147
Country         147
Anonymity       147
Google          104
Https           147
Last Checked    147
url             147
dtype: int64

## Sélectionner les proxies valides

In [15]:
url = "https://httpbin.org/ip"
good_proxies = set()
headers = browser_headers["Chrome"]
for proxy_url in https_proxies["url"]:
    proxies = {
        "http": proxy_url,
        "https": proxy_url,
    }
    
    try:
        response = requests.get(url, headers=headers, proxies=proxies, timeout=2)
        good_proxies.add(proxy_url)
        print(f"Proxy {proxy_url} OK, added to good_proxy list")
    except Exception:
        pass
    
    if len(good_proxies) >= 3:
        break

Proxy http://149.19.224.36:3128 OK, added to good_proxy list
Proxy http://3.17.13.205:8080 OK, added to good_proxy list
Proxy http://134.209.42.113:8899 OK, added to good_proxy list


# Application au scraping

1. Avec Selenium
2. Avec le module `requests` de Python

## Scraping avec Selenium

Si le site à scraper requiert l'utilisation de Javascript, vous n'aurez pas le choix et devrez utiliser Selenium comme montré dans le premier tuto. Vous pouvez faire une rotation de navigateurs mais le plus important est de faire une rotation de proxy.

In [16]:
from selenium import webdriver

for proxy_url in good_proxies:
    proxy = proxy_url.replace("http://", "")

    firefox_capabilities = webdriver.DesiredCapabilities.FIREFOX
    firefox_capabilities['marionette'] = True

    firefox_capabilities['proxy'] = {
        "proxyType": "MANUAL",
        "httpProxy": proxy,
        "sslProxy": proxy
    }

    driver = webdriver.Firefox(capabilities=firefox_capabilities)
    try:
        driver.get("https://httpbin.org/ip")
    except Exception:
        pass

## Scraping avec requests

Quand le site a scraper fonctionne sans Javascript, vous pouvez utiliser directement requests et faire une rotation de headers (dont `User-Agent`) et de proxies.

In [17]:
url = "https://httpbin.org/anything"
for browser, headers in browser_headers.items():
    print(f"\n\nUsing {browser} headers\n")
    for proxy_url in good_proxies:
        proxies = proxies = {
            "http": proxy_url,
            "https": proxy_url,
        }
        try:
            response = requests.get(url, headers=headers, proxies=proxies, timeout=2)
            print(response.json())
        except Exception:
            print(f"Proxy {proxy_url} failed, trying another one")



Using Chrome headers

{'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', 'Cache-Control': 'max-age=0', 'Host': 'httpbin.org', 'Sec-Fetch-Dest': 'document', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'same-origin', 'Sec-Fetch-User': '?1', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36', 'X-Amzn-Trace-Id': 'Root=1-61d1d299-64c75a5b5a49f42c7a9dbcfe'}, 'json': None, 'method': 'GET', 'origin': '149.19.224.36', 'url': 'https://httpbin.org/anything'}
Proxy http://3.17.13.205:8080 failed, trying another one
Proxy http://134.209.42.113:8899 failed, trying another one


Using Edge headers

{'args': {}, 'data': '', 'files

# Ouverture

Il existe des méthodes encore plus avancées de détection (et donc d'évasion de la détection) qui demandent des compétences plus avancées et donc difficilement accessibles à des débutants.

Si vous êtes intéressés et n'avez pas peur du challenge allez vous entrainer sur ce site : https://pixelscan.net/checkproxy?url=%2F